In [ ]:
# Author: Bridget Bittmann
# Date created: 08/16/2021
# Date last modified: 08/16/2021
# Purpose: This script extracts Sentinel 2 data for the Treasure Valley and 
# uses the data to calculate ET using the METRIC method. 

In [2]:
#Import geemap to access Google Earth Explorer data
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  

geemap package not installed. Installing ...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=f8FiIzwJeXDvjhaIBhK2VruNL_FZK5-aA7fiUmq2x8w&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjR_Se2YCeLbibtmZdCo6cRPiSDqpcQNmrU_bU3M5kPlCOB_MOQTgk

Successfully saved authorization token.


In [ ]:
# Algorithm:
# 1. Get huc value for watershed of interest (see table below)
# 3. Create a new feature with selected watershed
# 5. Union the watersheds (if necessary), export the watershed(s)
# 6. Get the Sentinel data
# 7. Clip to the area of the watershed
# 8. Export 

#--------------------------------#
#   Name           |    huc8     #
#--------------------------------#
#   Lower Boise    |  17050114   #
#--------------------------------#

In [3]:
wshd = ee.FeatureCollection("USGS/WBD/2017/HUC08")

lbrb_wshd = wshd.filterMetadata('name', 'equals', 'Lower Boise')

#print to map
map = emap.Map()
map.addLayer(ee.Image().paint(lbrb_wshd, 0, 2), {}, 'Lower Boise River Basin')
map.addLayerControl()
map

In [1]:
dataset = ee.ImageCollection("COPERNICUS/S2").filterDate("2015-01-01","2020-12-31")\
          .filterBounds(lbrb_wshd).median().divide(10000)

rgb = ["B4","B3","B2"] #creates bands variable
rgbVis =  {"min":0.0, "max":0.3,"bands":rgb} #sets thresholds for bands

#initialize map 
map = emap.Map()
map.centerObject(lbrb_wshd,7)
map.addLayer(dataset.clip(lbrb_wshd),rgbVis,"S2")

map.addLayerControl()
map



NameError: ignored

In [ ]:
# Function to mask clouds using the Sentinel-2 QA band
# param {ee.Image} image Sentinel-2 image
# return {ee.Image} cloud masked Sentinel-2 image

def maskS2clouds(image): 
  quality_band = image.select('QA60')

# Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

# Both flags should be set to zero, indicating clear conditions.
  mask = quality_band.bitwiseAnd(cloudBitMask).eq(0) and quality_band.bitwiseAnd(cirrusBitMask).eq(0),

  return image.updateMask(mask).divide(10000);

# Pre-filter to get less cloudy granules.
dataset = ee.ImageCollection('COPERNICUS/S2').filterDate('2015-01-01', '2020-12-31')\
                  .filterBounds(lbrb_wshd).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))\
                  .map(maskS2clouds).median()

#Initialize map for display
map_filt = emap.Map()
map_filt.centerObject(lbrb_wshd,7)
map_filt.addLayer(dataset.clip(lbrb_wshd), rgbVis, "S2")

map_filt.addLayerControl()
map